# Gemma models testing

## Preprocessing

In [2]:
from pdf_parser import *
import os

pdf_folder_path = "../data/PDF"
# pdf_files = [f for f in os.listdir(pdf_folder_path) if f.endswith('.pdf')]
pdf_files = ["24EM03352.pdf"]
image_save_folder_path = "../data/images"

pdf_text_image = {}
for pdf_file in pdf_files:
    pdf_text_image[os.path.splitext(pdf_file)[0]] = {"text":extract_all(os.path.join(pdf_folder_path,pdf_file))}
    pdf_images = extract_pdf2image(f"{pdf_folder_path}/{pdf_file}")
    pdf_text_image[os.path.splitext(pdf_file)[0]]["image"] = pdf_images

print(pdf_text_image.keys())
print(pdf_text_image)


2025-05-21 20:17:20,477 CropBox missing from /Page, defaulting to MediaBox


2025-05-21 20:17:21,156 CropBox missing from /Page, defaulting to MediaBox
2025-05-21 20:17:21,841 CropBox missing from /Page, defaulting to MediaBox


dict_keys(['24EM03352'])
{'24EM03352': {'text': "COPIE INTERNE 29/10/2024\n\nDr GOESSENS LAURENT \nRUE DE TOURNAI 42\n\nCentre d’Anatomie \n Pathologique H.U.B.\n\n7011 GHLIN\n\nRue Meylemeersch 90 - 1070 Anderlecht \nMijlemeerschstraat 90 – 1070 Anderlecht\n\nPATIENT :    \n  \nID :   \nRéf. Externe : 24MH9794 \n \nPrélevé le 23/07/2024 à 23/07/2024  \nReçu le 31/07/2024\n\nEXAMEN : 24EM03352 \n \nPrescripteur : Dr GOESSENS LAURENT\n\nDirectrice de Service \nPr Myriam Remmelink\n\nEquipe Médicale  \nDr Nicolas de Saint Aubain\n\nPr Nicky D’Haene \nDr Maria Gomez Galdon\n\nDr Chirine Khaled \nPr Denis Larsimont\n\nDr Laetitia Lebrun\n\nDr Calliope Maris \nPr Jean-Christophe Noël\n\nPr Isabelle Salmon \nDr Anne-Laure Trépant\n\nPr Laurine Verset\n\nConsultant (e) s\n\nRECHERCHE PAR « NEXT GENERATION SEQUENCING » DE \nMUTATIONS DANS 22 GENES IMPLIQUES DANS LES CANCERS\n\nDr Sarah Bouri \nDr Xavier Catteau \nDr Roland de Wind\n\nCOLORECTAUX ET PULMONAIRES\n\nDr Valérie Segers\n\n(COLON & 

In [3]:
with open("../prompt/system_prompt_metadata.txt", "r") as f:
    system_prompt = f.read()
print(system_prompt)

## Extraction de données d’un rapport de biopsie

Tu es un assistant médical intelligent spécialisé en oncologie. Ta tâche est de lire des documents PDF contenant des **rapports de biopsie** provenant de services d'oncologie hospitaliers et d’extraire toutes les informations médicales pertinentes et de les structurer.
Tu vas recevoir le document PDF en format image et en format textuel.
---

### Objectifs
1. Lire et comprendre le contenu d’un rapport de biopsie.
2. Extraire toutes les données cliniques demandées.
3. Retourner les données dans un format structuré et facilement exploitable.

---

### Informations à extraire

- Le numéro d'examen
- Le numéro du prélèvement
- Le panel de biopsie : colon and lung -> CLP, oncomine solid tumor -> OST, ovaire, endomètre, sein -> GP, thyroid -> TP, cancer -> CHP
- L'origine du prélèvement (lieu géographique)
- La qualité du séquencage
- Le pourcentage (%) de cellules (tumorales) analysées
---

### Format de sortie attendu

- Un dictionnaire str

## Model definition

When running on cpu, 38min inference time for 1 request

In [ ]:
from transformers import pipeline
import torch
from huggingface_hub import login

with open("../login.txt", "r") as f:
    token = f.read()
login(token) #token from huggingface.co necessary to use gemma3

model_name = "google/medgemma-4b-it"

pipe = pipeline(
    "image-text-to-text",
    model=model_name,
    torch_dtype=torch.bfloat16,
    device="cpu", #uses "cpu" here because the "cuda" requires 3 Go of VRAM
)

messages = [
    {
        "role": "system",
        "content": [{"type": "text", "text": system_prompt}]
    },
    {
        "role": "user",
        "content": (
            [{"type": "text", "text": pdf_text_image["24EM03352"]["text"]}]
            #+ [{"type": "image", "image": img} for img in pdf_text_image["24EM03352"]["image"]]
        )
    }
]

output = pipe(text=messages, max_new_tokens=400) #temperature=0
print(output[0]["generated_text"][-1]["content"])

Loading checkpoint shards: 100%|██████████| 2/2 [00:03<00:00,  1.73s/it]
Device set to use cpu
Setting `pad_token_id` to `eos_token_id`:1 for open-end generation.


```json
{
  "Examen": "24EM03352",
  "N° du prélèvement": "24MH9794 RF",
  "Panel": "COLP",
  "Origine du prélèvement": "Centre Hospitalier de Mouscron",
  "Type de prélèvement": "Adénocarcinome lieberkühnien",
  "Qualité du séquencage": "Optimale",
  "%" : "20",
  "Exons Non Contributif (coverage < 250x)*" : "AKT1 3, FGFR3 NM_000142 7, 9, 14, 16, 18, ALK NM_004304 22, 23, 25, KRAS NM_033360 2-


Show results

In [8]:
#save the output in json format
import json
output_json = {
    "pdf_file": pdf_file,
    "output": output[0]["generated_text"][-1]["content"]
}
with open("../out/medgemma3.json", "w") as f:
    json.dump(output_json, f, indent=4)
